In [1]:
%%local
!echo "anaconda" | kinit edill

Password for edill@DEMO.ANACONDA.COM: 


In [2]:
%%configure -f
{"driverMemory": "8G", 
 "driverCores": 4,
 "executorMemory": "8G",
 "executorCores": 4,
 "numExecutors": 3
}

In [3]:
from pyspark.sql import functions as F

Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log,Current session?
93,application_1561571912988_0189,pyspark,idle,Link,Link,✔


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

SparkSession available as 'spark'.


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [4]:
# raw
base = '/user/edill/anaconda-parsed-logs'
# cleaned
base = 's3://edill-data/cleaned-logs'
def date_to_path(datestring):
    y, m, d = datestring.split('.')[0].split('-')
    return '{base}/{year}/{month}/{year}-{month}-{day}.parquet'.format(
        base=base, year=y, month=m, day=d
    )

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [8]:
p = date_to_path('2016-10-28')
p

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

's3://edill-data/cleaned-logs/2016/10/2016-10-28.parquet'

In [9]:
sdf = spark.read.parquet(p)
sdf.head()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Row(time=datetime.datetime(2016, 10, 28, 17, 0), data_source=u'anaconda', pkg_name=u'anaconda-client', pkg_version=u'1.5.3', pkg_platform=u'linux-64', pkg_python=u'3.5', bytes=119691, day_name=u'Friday', hour=17, counts=1)

In [27]:
sdf.printSchema()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

root
 |-- time: timestamp (nullable = true)
 |-- data_source: string (nullable = true)
 |-- pkg_name: string (nullable = true)
 |-- pkg_version: string (nullable = true)
 |-- pkg_platform: string (nullable = true)
 |-- pkg_python: string (nullable = true)
 |-- counts: long (nullable = true)

In [25]:
sdf = spark.read.parquet('s3://edill-data/conda/hourly/2019/01/2019-01-01-fastparquet.parquet.SNAPPY')
sdf.head()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

An error occurred while calling o391.collectToPython.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 0 in stage 19.0 failed 4 times, most recent failure: Lost task 0.3 in stage 19.0 (TID 49, ip-10-0-2-143.ec2.internal, executor 7): java.lang.ArrayIndexOutOfBoundsException

Driver stacktrace:
	at org.apache.spark.scheduler.DAGScheduler.org$apache$spark$scheduler$DAGScheduler$$failJobAndIndependentStages(DAGScheduler.scala:1750)
	at org.apache.spark.scheduler.DAGScheduler$$anonfun$abortStage$1.apply(DAGScheduler.scala:1738)
	at org.apache.spark.scheduler.DAGScheduler$$anonfun$abortStage$1.apply(DAGScheduler.scala:1737)
	at scala.collection.mutable.ResizableArray$class.foreach(ResizableArray.scala:59)
	at scala.collection.mutable.ArrayBuffer.foreach(ArrayBuffer.scala:48)
	at org.apache.spark.scheduler.DAGScheduler.abortStage(DAGScheduler.scala:1737)
	at org.apache.spark.scheduler.DAGScheduler$$anonfun$handleTaskSetFailed$1.apply(DAGScheduler.scala:871)
	at org.a

In [8]:
sdf.select('pkg_channel').distinct().show()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

An error occurred while calling o93.showString.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 3 in stage 3.0 failed 4 times, most recent failure: Lost task 3.3 in stage 3.0 (TID 13, ip-10-0-2-132.ec2.internal, executor 3): java.lang.ArrayIndexOutOfBoundsException: 2205811
	at org.apache.parquet.column.values.bitpacking.ByteBitPackingLE$Packer16.unpack8Values(ByteBitPackingLE.java:8131)
	at org.apache.spark.sql.execution.datasources.parquet.VectorizedRleValuesReader.readNextGroup(VectorizedRleValuesReader.java:641)
	at org.apache.spark.sql.execution.datasources.parquet.VectorizedRleValuesReader.readInteger(VectorizedRleValuesReader.java:154)
	at org.apache.spark.sql.execution.datasources.parquet.VectorizedRleValuesReader.readIntegers(VectorizedRleValuesReader.java:481)
	at org.apache.spark.sql.execution.datasources.parquet.VectorizedColumnReader.readBatch(VectorizedColumnReader.java:168)
	at org.apache.spark.sql.execution.datasources.parquet.VectorizedParquet

In [24]:
sdf = (sdf.withColumn('date', F.to_date(F.col('timestamp')))
          .withColumn('hour', F.hour(F.col('timestamp'))))

In [15]:
sdf = sdf.filter(F.col('source').isin(['cloudflare_conda', 'cloudflare_repo_anaconda']) | 
           F.col('pkg_channel').isin(['conda-forge', 'bioconda', 'pyviz']))

In [27]:
cond = F.col('pkg_channel').isin(['conda-forge', 'bioconda', 'pyviz'])

In [30]:
sdf.select(F.col('pkg_channel'))[~cond] = 'anaconda'

TypeError: 'DataFrame' object does not support item assignment

In [35]:
cond

Column<(pkg_channel IN (conda-forge, bioconda, pyviz))>

In [61]:
sdf.select('pkg_channel').distinct().show()

+-----------+
|pkg_channel|
+-----------+
|       null|
+-----------+



In [ ]:
sdf.withColumn('pkg_channel2', F.when(sdf.select))

In [55]:
def replace(column, value):
    return when(column != value, column).otherwise(lit(None))


In [62]:
sdf.withColumn('pkg_channel2', 
               F.when(~sdf.select('pkg_python').isin(['33']), 'conda-forge').otherwise(F.lit(None))).head()

AttributeError: 'DataFrame' object has no attribute 'isin'